In [ ]:
import json
from rapidfuzz import fuzz
#funcion que esta en prod 
# === CONFIGURACIÓN ===
USAR_TOTAL_COMO_GATILLO = True

# === Función: concatenar campos clave ===
def concatenar_campos(ticket):
    claves = ["cuit", "puntoDeVenta", "fecha", "hora", "numeroDeTicket"]
    return " | ".join(str(ticket.get(c, "")).strip().lower() for c in claves)

# === Función: comparar campos clave ===
def comparar_campos_clave(t1, t2, umbral=90):
    cadena1 = concatenar_campos(t1)
    cadena2 = concatenar_campos(t2)
    similitud = fuzz.ratio(cadena1, cadena2)
    print(f"Similitud claves: {similitud:.2f}%")
    return similitud, similitud >= umbral

# === Función: comparar nombres de productos ===
def comparar_productos(t1, t2, umbral_individual=90):
    nombres1 = [p["nombre"].strip().lower() for p in t1.get("productos", []) if p.get("nombre")]
    nombres2 = [p["nombre"].strip().lower() for p in t2.get("productos", []) if p.get("nombre")]

    if not nombres1 or not nombres2:
        return 0, False

    # Comparar solo t1 contra t2 (una dirección)
    match = [max(fuzz.ratio(n1, n2) for n2 in nombres2) for n1 in nombres1]
    promedio = sum(match) / len(match)

    # Penalización por diferencia de cantidad
    diferencia_cantidad = abs(len(nombres1) - len(nombres2))
    penalizacion = diferencia_cantidad * 3
    score_final = max(0, promedio - penalizacion)

    return score_final, score_final >= umbral_individual

# === Verificación de duplicados entre dos tickets ===
def verificar_tickets(t1, t2, umbral_claves=90, umbral_productos=90):
    score_campos, pasa_claves = comparar_campos_clave(t1, t2, umbral_claves)

    # Descarte directo si las claves son 100% iguales
    #if score_campos == 100:
     #   return True, "claves 100% idénticas (duplicado seguro)"

    if pasa_claves:
        if USAR_TOTAL_COMO_GATILLO:
            try:
                total1 = float(str(t1.get("total", "")).replace(",", "").strip())
                total2 = float(str(t2.get("total", "")).replace(",", "").strip())
                diferencia_total = abs(total1 - total2)
                print(f"Diferencia total: {diferencia_total:.2f}")
            except:
                diferencia_total = float("inf")

            if diferencia_total < 5:
                if score_campos < umbral_claves + 2:
                    return True, f"claves + total (⚠ similitud justa {score_campos:.2f}%)"
                return True, "claves + total"
            else:
                score_productos, pasa_productos = comparar_productos(t1, t2, umbral_productos)
                if pasa_productos:
                    return True, f"productos ({score_productos:.2f}%)"
        else:
            if score_campos < umbral_claves + 2:
                return True, f"claves (⚠ similitud justa {score_campos:.2f}%)"
            return True, "claves"

    return False, None

# === Comparar todos contra todos (ordenadamente, solo hacia atrás) ===
def comparar_todos_contra_todos(tickets, umbral_claves=90, umbral_productos=90):
    duplicados = []
    unicos = []

    for i, nuevo_ticket in enumerate(tickets):
        es_duplicado = False

        for j in range(i):  # Comparar solo con anteriores
            ticket_anterior = tickets[j]
            son_duplicados, motivo = verificar_tickets(ticket_anterior, nuevo_ticket, umbral_claves, umbral_productos)

            if son_duplicados:
                duplicados.append({
                    "id_duplicado": i,
                    "id_original": j,
                    "numero_duplicado": nuevo_ticket.get("numeroDeTicket", ""),
                    "numero_original": ticket_anterior.get("numeroDeTicket", ""),
                    "cuit_duplicado": nuevo_ticket.get("cuit", ""),
                    "cuit_original": ticket_anterior.get("cuit", ""),
                    "fecha_duplicado": nuevo_ticket.get("fecha", ""),
                    "fecha_original": ticket_anterior.get("fecha", ""),
                    "motivo": motivo
                })
                es_duplicado = True
                break

        if not es_duplicado:
            unicos.append({
                "id": i,
                "numeroDeTicket": nuevo_ticket.get("numeroDeTicket", ""),
                "cuit": nuevo_ticket.get("cuit", ""),
                "fecha": nuevo_ticket.get("fecha", "")
            })

    return duplicados, unicos


In [ ]:
import json
import pandas as pd
from datetime import datetime, timedelta
from rapidfuzz import fuzz
from collections import defaultdict
from tqdm import tqdm
#funcion a probar

def procesar_tickets(ruta, umbral_productos=90, umbral_claves=85, validar_fecha=True, salida_csv="tickets_clasificados.csv"):
    def validar_cuit(cuit):
        try:
            cuit = cuit.replace("-", "")
            if len(cuit) != 11 or not cuit.isdigit():
                return False
            mult = [5, 4, 3, 2, 7, 6, 5, 4, 3, 2]
            suma = sum(int(cuit[i]) * mult[i] for i in range(10))
            resto = suma % 11
            verificador = 11 - resto if resto != 0 else 0
            return verificador == int(cuit[-1])
        except:
            return False

    def normalizar_pv(pv):
        reemplazos = {'3': '8', '8': '8', '1': '7', '7': '7'}
        return ''.join(reemplazos.get(c, c) for c in pv)

    def filtrar_por_fecha(tickets):
        hoy = datetime.today().date()
        ayer = hoy - timedelta(days=1)
        tickets_validos, moderacion, descartados = [], [], []

        for t in tickets:
            fecha_str = t.get("fecha", "")
            try:
                fecha_ticket = datetime.strptime(fecha_str, "%d/%m/%Y").date()
                if fecha_ticket in [hoy, ayer]:
                    tickets_validos.append((t, "válido", "-"))
                else:
                    descartados.append((t, "descartado", "fecha fuera de rango"))
            except:
                if fuzz.partial_ratio(fecha_str, hoy.strftime("%d/%m/%Y")) > 80 or \
                   fuzz.partial_ratio(fecha_str, ayer.strftime("%d/%m/%Y")) > 80:
                    moderacion.append((t, "moderación", "fecha dudosa (OCR)"))
                else:
                    descartados.append((t, "descartado", "fecha inválida"))

        if not validar_fecha:
            return [(t, "válido", "-") for t in tickets], [], []

        return tickets_validos + moderacion, descartados, moderacion

    def comparar_por_claves(t1, t2):
        try:
            f1 = datetime.strptime(t1["fecha"], "%d/%m/%Y")
            f2 = datetime.strptime(t2["fecha"], "%d/%m/%Y")
            if f1 != f2:
                return False, 0.0
        except:
            return False, 0.0

        claves = ["numeroDeTicket", "hora"]
        sim_sum = 0
        for campo in claves:
            if campo in t1 and campo in t2:
                sim = fuzz.token_sort_ratio(str(t1[campo]), str(t2[campo]))
                sim_sum += sim
        promedio = sim_sum / len(claves)
        return promedio >= umbral_claves, promedio

    def comparar_por_productos(t1, t2):
        def extraer_nombres_relevantes(productos):
            if len(productos) == 0:
                return []
            if len(productos) <= 4:
                return [p["nombre"] for p in productos]
            return [productos[i]["nombre"] for i in [0, 1, 2, -1] if i < len(productos)]

        productos1 = extraer_nombres_relevantes(t1.get("productos", []))
        productos2 = extraer_nombres_relevantes(t2.get("productos", []))
        return fuzz.token_sort_ratio(" ".join(productos1), " ".join(productos2))

    def agrupar_tickets_por_cuit_pv(tickets):
        agrupados = defaultdict(lambda: defaultdict(list))
        for ticket, estado, motivo in tickets:
            cuit = ticket.get("cuit", "").strip()
            if not validar_cuit(cuit):
                motivo = "CUIT inválido"
                estado = "advertencia"
            pv = normalizar_pv(ticket.get("puntoDeVenta", ""))
            agrupados[cuit][pv].append((ticket, estado, motivo))
        return agrupados

    with open(ruta, "r", encoding="utf-8") as f:
        raw_tickets = [json.loads(line) for line in f]

    filtrados, descartados, _ = filtrar_por_fecha(raw_tickets)
    agrupados = agrupar_tickets_por_cuit_pv(filtrados)

    registros = []
    progreso = tqdm(total=sum(len(agrupados[c]) for c in agrupados), desc="🔍 Analizando grupos", ncols=80)

    for cuit in agrupados:
        for pv in agrupados[cuit]:
            grupo = agrupados[cuit][pv]
            locales = []
            for ticket, estado, motivo in grupo:
                es_duplicado = False
                duplicado_de = "-"
                similitud = "-"

                for otro, _, _ in locales:
                    claves_ok, sim_claves = comparar_por_claves(ticket, otro)
                    if claves_ok:
                        es_duplicado = True
                        estado = "duplicado"
                        motivo = "duplicado por claves"
                        duplicado_de = otro["numeroDeTicket"]
                        similitud = round(sim_claves, 2)
                        break

                if not es_duplicado:
                    for otro, _, _ in locales:
                        sim_prod = comparar_por_productos(ticket, otro)
                        if sim_prod >= umbral_productos:
                            es_duplicado = True
                            estado = "duplicado"
                            motivo = "duplicado por productos"
                            duplicado_de = otro["numeroDeTicket"]
                            similitud = round(sim_prod, 2)
                            break

                if not es_duplicado:
                    estado = "único"
                    motivo = "-"
                    similitud = "-"
                    locales.append((ticket, estado, motivo))

                registros.append({
                    "numeroDeTicket": ticket.get("numeroDeTicket", ""),
                    "estado": estado,
                    "motivo": motivo,
                    "duplicadoDe": duplicado_de,
                    "similitud": similitud,
                    "fecha": ticket.get("fecha", ""),
                    "cuit": ticket.get("cuit", ""),
                    "nombreComercio": ticket.get("nombreComercio", "")
                })
            progreso.update(1)

    for ticket, estado, motivo in descartados:
        registros.append({
            "numeroDeTicket": ticket.get("numeroDeTicket", ""),
            "estado": estado,
            "motivo": motivo,
            "duplicadoDe": "-",
            "similitud": "-",
            "fecha": ticket.get("fecha", ""),
            "cuit": ticket.get("cuit", ""),
            "nombreComercio": ticket.get("nombreComercio", "")
        })

    df = pd.DataFrame(registros)
    df.to_csv(salida_csv, index=False)
    print(f"\n✅ Archivo CSV generado: {salida_csv}")
    print(f"✔ Totales: {len(df)} tickets clasificados.")


ModuleNotFoundError: No module named 'rapidfuzz'